In [1]:
import pandas as pd
import xarray as xr

from scipy import optimize as opt
from scipy import stats

In [2]:
#fname = sys.argv[1]
path = "../nctemp/*"
ds = xr.open_mfdataset( path, parallel = True )

In [14]:
ds = xr.open_mfdataset( path, parallel = True )
ds.drop("XTIME_bnds")

<xarray.Dataset>
Dimensions:   (south_north: 112, west_east: 132, AÑO: 31, DURACION: 5)
Coordinates:
    XLONG     (south_north, west_east) float32 dask.array<chunksize=(112, 132), meta=np.ndarray>
    XLAT      (south_north, west_east) float32 dask.array<chunksize=(112, 132), meta=np.ndarray>
  * DURACION  (DURACION) int64 1 2 3 4 5
  * AÑO       (AÑO) int64 1985 1986 1987 1988 1989 ... 2011 2012 2013 2014 2015
Dimensions without coordinates: south_north, west_east
Data variables:
    Pcp       (AÑO, DURACION, south_north, west_east) float32 dask.array<chunksize=(1, 5, 112, 132), meta=np.ndarray>

In [9]:
i = 1
df = ds.sel( DURACION = [i] ).to_dataframe().sort_values(
        ["south_north", "west_east", "Pcp"], ascending = [True, True, False] )
df["m"] = ( list( range(1, ds["AÑO"].count().values + 1 ) )
        * ( ds["west_east"].max().values + 1 )
        * ( ds["south_north"].max().values + 1 ) )

ValueError: Length of values (458304) does not match length of index (916608)

In [7]:
ds_i = []
for i in ds["DURACION"].values:
    print(i)
    # Separamos por duración, reordenamos la intensidad
    # de precipitación de cada celda de mayor a menor.
    df = ds.sel( DURACION = [i] ).to_dataframe().sort_values(
        ["south_north", "west_east", "Pcp"], ascending = [True, True, False] )
    # Calculamos el número de orden de mayor a menor.
    df["m"] = ( list( range(1, ds["AÑO"].count().values + 1 ) )
        * ( ds["west_east"].max().values + 1 )
        * ( ds["south_north"].max().values + 1 ) )
    # Calculamos el tiempo de retorno.
    df["TIEMPO_RETORNO"] = ( ds["AÑO"].count().values + 1 ) / df["m"]
    # Calculamos la probabilidad de excedencia.
    df["PROBABILIDAD"] = 1/ df["TIEMPO_RETORNO"]
    ds_i.append( df.drop("m", axis = 1).to_xarray() )

1


ValueError: Length of values (458304) does not match length of index (916608)

In [27]:
path = "../temp/SAM/*nc"
ds = xr.open_mfdataset( path, parallel = True )
ds

,Array,Chunk
Bytes,2.48 MiB,60.55 kiB
Shape,"(42, 5, 50, 62)","(1, 5, 50, 62)"
Count,126 Tasks,42 Chunks
Type,float32,numpy.ndarray


In [33]:
len( list( range(1, ds["AÑO"].count().values + 1 ) ))

42

In [42]:
len( list( range(1, ds["AÑO"].count().values + 1 ) )
        * ( ds["longitude"].count().values + 0 )
        * ( ds["latitude"].count().values + 0 ) )

130200

In [2]:
path = "../results/prec_hist_hist_tretorno.nc"

df = xr.open_dataset( path ).to_dataframe()

# Quitamos la intensidad, año, probabilidad, tiempo de retorno, y agregamos
# columnas para los parámetros de la distribución.
df_2 = df.copy().drop( ["PROBABILIDAD", "AÑO"], axis = 1  #)
    ).reset_index( "TIEMPO_RETORNO" )
#df_2 = df_2.reset_index( "DURACION" )
df_3 = df_2.copy(
    #).reset_index( "TIEMPO_RETORNO"
    ).drop( ["INTENSIDAD", "TIEMPO_RETORNO"], axis = 1).groupby(
    ["south_north", "west_east", "DURACION"] ).mean()
cols = ["GEV_C", "GEV_LOC", "GEV_SCALE", "KTEST_P"]
df_3[ cols ] = None
t = ( ( [5, 10, 25, 50, 100, 200, 500, 1000] + [None] * 23 )
    * df_2.index.get_level_values("west_east").unique().shape[0]
    * df_2.index.get_level_values("south_north").unique().shape[0]
    * df_2.index.get_level_values("DURACION").unique().shape[0] )
#df_2["DURACION"] = d
df_2["TIEMPO_RETORNO"] = t
df_2 = df_2.dropna().set_index( "TIEMPO_RETORNO", append = True )

df_2

LONGITUD    LATITUD  \
south_north west_east DURACION TIEMPO_RETORNO                          
0           0         1        5.0            -100.453476  18.135178   
                               10.0           -100.453476  18.135178   
                               25.0           -100.453476  18.135178   
                               50.0           -100.453476  18.135178   
                               100.0          -100.453476  18.135178   
...                                                   ...        ...   
111         131       120      50.0            -97.333252  20.628967   
                               100.0           -97.333252  20.628967   
                               200.0           -97.333252  20.628967   
                               500.0           -97.333252  20.628967   
                               1000.0          -97.333252  20.628967   

                                               INTENSIDAD  
south_north west_east DURACION TIEMPO_RETORNO              
0           0         1        5.0                    0.0  
                               10.0                   0.0  
                               25.0                   0.0  
                               50.0                   0.0  
                               100.0                  0.0  
...                                                   ...  
111         131       120      50.0                   0.0  
                               100.0                  0.0  
                               200.0                  0.0  
                               500.0                  0.0  
                               1000.0                 0.0  

[2601984 rows x 3 columns]

In [3]:


# Iteramos para cada celda y duración.
#for i in df_3.index.get_level_values("south_north").unique():
for i in range(0, 3):
    print(f"Calculando coordenada y #{i}...")
    # for j in df_3.index.get_level_values("west_east").unique():
    for j in range(0, 3):
        for k in df_3.index.get_level_values("DURACION").unique():
            # ajustamos la distribución de valores extremos.
            params = stats.genextreme.fit( df.loc[ (i, j, k), "INTENSIDAD" ] )
            # Hacemos la prueba Kolmogorov Smirnoff.
            pvalue = stats.kstest( df.loc[ (i, j, k), "INTENSIDAD" ],
                stats.genextreme(*params).cdf ).pvalue
            df_3.loc[ (i, j, k), cols ] = [*params] + [pvalue]
            # Calculamos las intensidades de acuerdo con la distribución.
            df_2.loc[ (i, j, k), "INTENSIDAD"] = stats.genextreme(
                *df_3.loc[ (i, j, k), cols[:-1] ] ).isf(
                1 / df_2.loc[ (i, j, k) ].index.get_level_values(
                "TIEMPO_RETORNO") )

Calculando coordenada y #0...


/opt/homebrew/Caskroom/miniforge/base/envs/gv/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:406: RuntimeWarning: invalid value encountered in true_divide
  return m3 / np.power(m2, 1.5)


Calculando coordenada y #1...
Calculando coordenada y #2...


In [5]:
ds

<xarray.Dataset>
Dimensions:      (south_north: 112, west_east: 132, DURACION: 22)
Coordinates:
  * south_north  (south_north) int64 0 1 2 3 4 5 6 ... 106 107 108 109 110 111
  * west_east    (west_east) int64 0 1 2 3 4 5 6 ... 125 126 127 128 129 130 131
  * DURACION     (DURACION) int64 1 2 3 4 5 6 8 10 ... 42 48 60 72 84 96 108 120
    LONGITUD     (south_north, west_east, DURACION) float32 -100.5 ... -97.33
    LATITUD      (south_north, west_east, DURACION) float32 18.14 ... 20.63
Data variables:
    GEV_C        (south_north, west_east, DURACION) object -0.774 ... None
    GEV_LOC      (south_north, west_east, DURACION) object -5.698e-23 ... None
    GEV_SCALE    (south_north, west_east, DURACION) object 5.136e-23 ... None
    KTEST_P      (south_north, west_east, DURACION) object 8.369e-13 ... None

In [6]:
# Guardamos los valores de intensidad de las curvas IDF.
ds = df_2.to_xarray().set_coords( ["LONGITUD", "LATITUD"] )
ds["LONGITUD"] = ds["LONGITUD"].isel( {"DURACION": 1, "TIEMPO_RETORNO": 1}
    ).drop( ["DURACION", "TIEMPO_RETORNO"] )
ds["LATITUD"] = ds["LATITUD"].isel( {"DURACION": 1, "TIEMPO_RETORNO": 1}
    ).drop( ["DURACION", "TIEMPO_RETORNO"] )
ds.to_netcdf("b.nc")

# Guardamos los parámetros de la distribución GEV para las curvas IDF.
ds = df_3.to_xarray().set_coords( ["LONGITUD", "LATITUD"] )
ds["LONGITUD"] = ds["LONGITUD"].isel( {"DURACION": 1} ).drop( ["DURACION"] )
ds["LATITUD"]  = ds["LATITUD"].isel(  {"DURACION": 1} ).drop( ["DURACION"] )
ds.to_netcdf("a.nc")

In [19]:
df_2.to_xarray().set_coords( ["LONGITUD", "LATITUD"] ).drop_vars(["south_north", "west_east"])

<xarray.Dataset>
Dimensions:         (DURACION: 22, TIEMPO_RETORNO: 8, south_north: 112, west_east: 132)
Coordinates:
  * DURACION        (DURACION) int64 1 2 3 4 5 6 8 10 ... 48 60 72 84 96 108 120
  * TIEMPO_RETORNO  (TIEMPO_RETORNO) float64 5.0 10.0 25.0 ... 200.0 500.0 1e+03
    LONGITUD        (south_north, west_east, DURACION, TIEMPO_RETORNO) float32 ...
    LATITUD         (south_north, west_east, DURACION, TIEMPO_RETORNO) float32 ...
Dimensions without coordinates: south_north, west_east
Data variables:
    INTENSIDAD      (south_north, west_east, DURACION, TIEMPO_RETORNO) float32 ...

In [116]:
path = "../temp/*.nc"

def pre(ds):
    ds = ( ds.expand_dims( ESTADISTICA = int( 
        #ds["XLAT"].encoding["source"][-4:-3] )
        1 )
    
        ).rename_dims( { "XTIME":"MES" } 
        ).rename_vars( { "XLAT": "LATITUD", "XLONG": "LONGITUD",
        "XTIME": "MES", "Pcp": "PRECIPITACION" }
        ).drop_vars( ["XTIME_bnds"] )
        )
    return ds

ds = xr.open_mfdataset( path,
            combine = "nested", 
            concat_dim = "ESTADISTICA",
            parallel = True, preprocess = pre
            )

ds["MES"] = range(1, 13)
ds["ESTADISTICA"] = range(1, 5)
#ds = ds.swap_dims( ["south_north", "west_east", "MES", "ESTADISTICA"] )

ds.to_dataframe().reorder_levels( ["south_north",
    "west_east", "MES", "ESTADISTICA"] ).sort_index(
    ).to_xarray().set_coords( ["LONGITUD", "LATITUD"] )

<xarray.Dataset>
Dimensions:        (south_north: 112, west_east: 132, MES: 12, ESTADISTICA: 4)
Coordinates:
  * south_north    (south_north) int64 0 1 2 3 4 5 6 ... 106 107 108 109 110 111
  * west_east      (west_east) int64 0 1 2 3 4 5 6 ... 126 127 128 129 130 131
  * MES            (MES) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * ESTADISTICA    (ESTADISTICA) int64 1 2 3 4
    LONGITUD       (south_north, west_east, MES, ESTADISTICA) float32 -100.5 ...
    LATITUD        (south_north, west_east, MES, ESTADISTICA) float32 18.14 ....
Data variables:
    PRECIPITACION  (south_north, west_east, MES, ESTADISTICA) float32 0.0 ......